# SQL with Chinook Data set

In [1]:
import sqlite3
import pandas as pd
conn = sqlite3.connect('chinook.db')
cursor = conn.cursor()

## Start running queries

Columns: Track ID, Track Name, Artist Name, Type, Price Quantity<br>
Tables: Track, Media_Type, Invoice_Line, Artist, Album

In [3]:
q = '''
SELECT track.track_id, track.name track_name, artist.name artist_name, med.name track_type, inv.unit_price unit_price, inv.quantity
FROM invoice_line inv
INNER JOIN track ON track.track_id = inv.track_id
INNER JOIN media_type med ON track.media_type_id = med.media_type_id
INNER JOIN album ON album.album_id = track.album_id
INNER JOIN artist ON artist.artist_id = album.artist_id
WHERE inv.invoice_id = 4
;'''

pd.read_sql_query(q, conn)

,track_id,track_name,artist_name,track_type,unit_price,quantity
0,3448,"Lamentations of Jeremiah, First Set \ Incipit ...",The King's Singers,Protected AAC audio file,0.99,1
1,2560,Violent Pornography,System Of A Down,MPEG audio file,0.99,1
2,3336,War Pigs,Cake,Purchased AAC audio file,0.99,1
3,829,Let's Get Rocked,Def Leppard,MPEG audio file,0.99,1
4,1872,Attitude,Metallica,MPEG audio file,0.99,1
5,748,Dealer,Deep Purple,MPEG audio file,0.99,1
6,1778,You're What's Happening (In The World Today),Marvin Gaye,MPEG audio file,0.99,1
7,2514,Spoonman,Soundgarden,MPEG audio file,0.99,1


Columns: Album, Artist, Tracks_Purchased<br>
Tables: Invlice_Line, Track, Album, Artist

In [3]:
q = '''
SELECT sub.al_name album, sub.ar_name artist, SUM(il.quantity) tracks_purchased
FROM invoice_line il
INNER JOIN (SELECT t.track_id, ar.name ar_name, al.title al_name
            FROM track t
            INNER JOIN album al ON al.album_id = t.album_id
            INNER JOIN artist ar ON al.artist_id = ar.artist_id) as sub
            ON sub.track_id = il.track_id
GROUP BY album
ORDER BY tracks_purchased DESC
LIMIT 5
;'''

pd.read_sql_query(q, conn)

,album,artist,tracks_purchased
0,Are You Experienced?,Jimi Hendrix,187
1,Faceless,Godsmack,96
2,Mezmerize,System Of A Down,93
3,Get Born,JET,90
4,The Doors,The Doors,83


### SQL Concatination

Columns: Employee Name, Employee Title, Supervisor Name, Supervisor Title <br>
Tables: Employee joined with itself

In [4]:
q = '''
SELECT 
e1.first_name || " " || e1.last_name employee_name,
e1.title employee_title,
e2.first_name || " " || e2.last_name supervisor_name,
e2.title supervisor_title
FROM employee e1
LEFT JOIN employee e2 
ON e1.reports_to = e2.employee_id
ORDER BY employee_name
;'''

pd.read_sql_query(q, conn)

,employee_name,employee_title,supervisor_name,supervisor_title
0,Andrew Adams,General Manager,None,None
1,Jane Peacock,Sales Support Agent,Nancy Edwards,Sales Manager
2,Laura Callahan,IT Staff,Michael Mitchell,IT Manager
3,Margaret Park,Sales Support Agent,Nancy Edwards,Sales Manager
4,Michael Mitchell,IT Manager,Andrew Adams,General Manager
5,Nancy Edwards,Sales Manager,Andrew Adams,General Manager
6,Robert King,IT Staff,Michael Mitchell,IT Manager
7,Steve Johnson,Sales Support Agent,Nancy Edwards,Sales Manager


Using % and LIKE for pattern matching

In [5]:
q = '''
SELECT first_name, last_name, phone
FROM customer
WHERE first_name LIKE "%Belle%"
;'''

pd.read_sql_query(q, conn)

,first_name,last_name,phone
0,Isabelle,Mercier,+33 03 80 73 66 99


CASE Aliases

In [6]:
q = '''
SELECT 
c.first_name || " " || c.last_name customer_name,
i.purchases number_of_purchases,
i.total_spent,
CASE
    WHEN total_spent < 40 THEN "small spender"
    WHEN total_spent > 100 THEN "big spender"
    ELSE "regular"
END
    AS customer_category
FROM customer c INNER JOIN (SELECT SUM(total) as total_spent, COUNT(*) as purchases, customer_id
                            FROM invoice
                            GROUP BY customer_id
                            ) as i
                ON c.customer_id = i.customer_id
ORDER BY customer_name
;'''

pd.read_sql_query(q, conn)

,customer_name,number_of_purchases,total_spent,customer_category
0,Aaron Mitchell,8,70.29,regular
1,Alexandre Rocha,10,69.30,regular
2,Astrid Gruber,9,69.30,regular
3,Bjørn Hansen,9,72.27,regular
4,Camille Bernard,9,79.20,regular
5,Daan Peeters,7,60.39,regular
6,Dan Miller,12,95.04,regular
7,Diego Gutiérrez,5,39.60,small spender
8,Dominique Lefebvre,9,72.27,regular
9,Eduardo Martins,12,60.39,regular


## Begin Working on Style and Readability for SQL Queries

Naming subqueries with WITH and making sure that Spacing services readability in code style

In [7]:
q = '''
WITH playlist_info AS
    (
    SELECT p.playlist_id, p.name playlist_name, t.name track_name, (t.milliseconds / 1000) seconds
    FROM playlist p 
    LEFT JOIN playlist_track pt ON p.playlist_id = pt.playlist_id
    LEFT JOIN track t ON pt.track_id = t.track_id
    )
SELECT playlist_id, playlist_name, COUNT(track_name) number_of_tracks, SUM(seconds) length_seconds
FROM playlist_info
GROUP BY playlist_id, playlist_name
ORDER BY playlist_id
;'''

pd.read_sql_query(q, conn)

,playlist_id,playlist_name,number_of_tracks,length_seconds
0,1,Music,3290,876049.0
1,2,Movies,0,NaN
2,3,TV Shows,213,500987.0
3,4,Audiobooks,0,NaN
4,5,90’s Music,1477,397970.0
5,6,Audiobooks,0,NaN
6,7,Movies,0,NaN
7,8,Music,3290,876049.0
8,9,Music Videos,1,294.0
9,10,TV Shows,213,500987.0


In [8]:
q = '''
SELECT p.playlist_id, p.name playlist_name, t.name track_name, (t.milliseconds / 1000) seconds
    FROM playlist p 
    LEFT JOIN playlist_track pt ON p.playlist_id = pt.playlist_id
    INNER JOIN track t ON pt.track_id = t.track_id
;'''

pd.read_sql_query(q, conn)

,playlist_id,playlist_name,track_name,seconds
0,1,Music,For Those About To Rock (We Salute You),343
1,1,Music,Balls to the Wall,342
2,1,Music,Fast As a Shark,230
3,1,Music,Restless and Wild,252
4,1,Music,Princess of the Dawn,375
5,1,Music,Put The Finger On You,205
6,1,Music,Let's Get It Up,233
7,1,Music,Inject The Venom,210
8,1,Music,Snowballed,203
9,1,Music,Evil Walks,263


### Creating and using VIEWs

In [9]:
cursor.execute('DROP VIEW customer_gt_90_dollars')
cursor.execute('DROP VIEW customer_usa')

In [10]:
q = '''
CREATE VIEW customer_usa AS 
     SELECT * FROM customer
     WHERE country = "USA"
;'''
cursor.execute(q)

Make a View that contains all customer data for customers with more than $90 spent

In [11]:
q1 = '''
CREATE VIEW customer_gt_90_dollars AS
    SELECT c.*
    FROM customer c
    INNER JOIN invoice i ON c.customer_id = i.customer_id
    GROUP BY c.customer_id
    HAVING SUM(i.total) > 90;
'''

q2 = '''SELECT *
FROM customer_gt_90_dollars
;'''

cursor.execute(q1)
pd.read_sql_query(q2, conn)

,customer_id,first_name,last_name,company,address,city,state,country,postal_code,phone,fax,email,support_rep_id
0,1,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,+55 (12) 3923-5555,+55 (12) 3923-5566,luisg@embraer.com.br,3
1,3,François,Tremblay,None,1498 rue Bélanger,Montréal,QC,Canada,H2G 1A7,+1 (514) 721-4711,None,ftremblay@gmail.com,3
2,5,František,Wichterlová,JetBrains s.r.o.,Klanova 9/506,Prague,None,Czech Republic,14700,+420 2 4172 5555,+420 2 4172 5555,frantisekw@jetbrains.com,4
3,6,Helena,Holý,None,Rilská 3174/6,Prague,None,Czech Republic,14300,+420 2 4177 0449,None,hholy@gmail.com,5
4,13,Fernanda,Ramos,None,Qe 7 Bloco G,Brasília,DF,Brazil,71020-677,+55 (61) 3363-5547,+55 (61) 3363-7855,fernadaramos4@uol.com.br,4
5,17,Jack,Smith,Microsoft Corporation,1 Microsoft Way,Redmond,WA,USA,98052-8300,+1 (425) 882-8080,+1 (425) 882-8081,jacksmith@microsoft.com,5
6,20,Dan,Miller,None,541 Del Medio Avenue,Mountain View,CA,USA,94040-111,+1 (650) 644-3358,None,dmiller@comcast.com,4
7,21,Kathy,Chase,None,801 W 4th Street,Reno,NV,USA,89503,+1 (775) 223-7665,None,kachase@hotmail.com,5
8,22,Heather,Leacock,None,120 S Orange Ave,Orlando,FL,USA,32801,+1 (407) 999-7788,None,hleacock@gmail.com,4
9,30,Edward,Francis,None,230 Elgin Street,Ottawa,ON,Canada,K2P 1L7,+1 (613) 234-3322,None,edfrancis@yachoo.ca,3


Query All Customers who are in the US or who spent more than $90

In [12]:
q = '''
SELECT *
FROM customer_usa

UNION

SELECT *
FROM customer_gt_90_dollars
;'''

pd.read_sql_query(q, conn)

,customer_id,first_name,last_name,company,address,city,state,country,postal_code,phone,fax,email,support_rep_id
0,1,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,+55 (12) 3923-5555,+55 (12) 3923-5566,luisg@embraer.com.br,3
1,3,François,Tremblay,None,1498 rue Bélanger,Montréal,QC,Canada,H2G 1A7,+1 (514) 721-4711,None,ftremblay@gmail.com,3
2,5,František,Wichterlová,JetBrains s.r.o.,Klanova 9/506,Prague,None,Czech Republic,14700,+420 2 4172 5555,+420 2 4172 5555,frantisekw@jetbrains.com,4
3,6,Helena,Holý,None,Rilská 3174/6,Prague,None,Czech Republic,14300,+420 2 4177 0449,None,hholy@gmail.com,5
4,13,Fernanda,Ramos,None,Qe 7 Bloco G,Brasília,DF,Brazil,71020-677,+55 (61) 3363-5547,+55 (61) 3363-7855,fernadaramos4@uol.com.br,4
5,16,Frank,Harris,Google Inc.,1600 Amphitheatre Parkway,Mountain View,CA,USA,94043-1351,+1 (650) 253-0000,+1 (650) 253-0000,fharris@google.com,4
6,17,Jack,Smith,Microsoft Corporation,1 Microsoft Way,Redmond,WA,USA,98052-8300,+1 (425) 882-8080,+1 (425) 882-8081,jacksmith@microsoft.com,5
7,18,Michelle,Brooks,None,627 Broadway,New York,NY,USA,10012-2612,+1 (212) 221-3546,+1 (212) 221-4679,michelleb@aol.com,3
8,19,Tim,Goyer,Apple Inc.,1 Infinite Loop,Cupertino,CA,USA,95014,+1 (408) 996-1010,+1 (408) 996-1011,tgoyer@apple.com,3
9,20,Dan,Miller,None,541 Del Medio Avenue,Mountain View,CA,USA,94040-111,+1 (650) 644-3358,None,dmiller@comcast.com,4


Query the number of customers with large purchases in the us who went to each sales rep.

***Wanted to include sales reps that had 0 such customers as well***

In [13]:
q = '''
WITH customers_usa_gt_90 AS
    (SELECT *
     FROM customer_gt_90_dollars
     
     INTERSECT
     
     SELECT *
     FROM customer_usa)
     
SELECT 
    e.first_name ||' ' || e.last_name employee_name,
    COUNT(c.customer_id) customers_usa_gt_90_dollars
FROM
    employee e
    LEFT JOIN
    customers_usa_gt_90 as c
    ON c.support_rep_id = e.employee_id
GROUP BY e.employee_id HAVING e.title = 'Sales Support Agent'
;'''

pd.read_sql_query(q, conn)

,employee_name,customers_usa_gt_90_dollars
0,Jane Peacock,0
1,Margaret Park,2
2,Steve Johnson,2


Just checking out the **employee** table

In [14]:
pd.read_sql_query('SELECT * FROM employee', conn)

,employee_id,last_name,first_name,title,reports_to,birthdate,hire_date,address,city,state,country,postal_code,phone,fax,email
0,1,Adams,Andrew,General Manager,NaN,1962-02-18 00:00:00,2016-08-14 00:00:00,11120 Jasper Ave NW,Edmonton,AB,Canada,T5K 2N1,+1 (780) 428-9482,+1 (780) 428-3457,andrew@chinookcorp.com
1,2,Edwards,Nancy,Sales Manager,1.0,1958-12-08 00:00:00,2016-05-01 00:00:00,825 8 Ave SW,Calgary,AB,Canada,T2P 2T3,+1 (403) 262-3443,+1 (403) 262-3322,nancy@chinookcorp.com
2,3,Peacock,Jane,Sales Support Agent,2.0,1973-08-29 00:00:00,2017-04-01 00:00:00,1111 6 Ave SW,Calgary,AB,Canada,T2P 5M5,+1 (403) 262-3443,+1 (403) 262-6712,jane@chinookcorp.com
3,4,Park,Margaret,Sales Support Agent,2.0,1947-09-19 00:00:00,2017-05-03 00:00:00,683 10 Street SW,Calgary,AB,Canada,T2P 5G3,+1 (403) 263-4423,+1 (403) 263-4289,margaret@chinookcorp.com
4,5,Johnson,Steve,Sales Support Agent,2.0,1965-03-03 00:00:00,2017-10-17 00:00:00,7727B 41 Ave,Calgary,AB,Canada,T3B 1Y7,1 (780) 836-9987,1 (780) 836-9543,steve@chinookcorp.com
5,6,Mitchell,Michael,IT Manager,1.0,1973-07-01 00:00:00,2016-10-17 00:00:00,5827 Bowness Road NW,Calgary,AB,Canada,T3B 0C5,+1 (403) 246-9887,+1 (403) 246-9899,michael@chinookcorp.com
6,7,King,Robert,IT Staff,6.0,1970-05-29 00:00:00,2017-01-02 00:00:00,590 Columbia Boulevard West,Lethbridge,AB,Canada,T1K 5N8,+1 (403) 456-9986,+1 (403) 456-8485,robert@chinookcorp.com
7,8,Callahan,Laura,IT Staff,6.0,1968-01-09 00:00:00,2017-03-04 00:00:00,923 7 ST NW,Lethbridge,AB,Canada,T1H 1Y8,+1 (403) 467-3351,+1 (403) 467-8772,laura@chinookcorp.com


Query customer name and total from india. <br>
Use two subqueries. <br>

*Mistake that I made was to join invoice to customer ID for customer_total. No Join was necessary*

In [15]:
q = '''
WITH 
    customers_india AS
        (
         SELECT *
         FROM customer
         WHERE country = 'India'
        ),
    customer_total AS
        (
         SELECT customer_id, SUM(total) total
         FROM invoice
         GROUP BY customer_id
        )


        
SELECT 
    ci.first_name || ' ' || ci.last_name customer_name,
    ct.total total_purchases
FROM 
    customers_india ci
INNER JOIN customer_total ct ON ci.customer_id = ct.customer_id
ORDER BY customer_name
;'''

pd.read_sql_query(q, conn)

,customer_name,total_purchases
0,Manoj Pareek,111.87
1,Puja Srivastava,71.28


Find the best customer from each country

In [36]:
q = '''
WITH
    customer_country_purchases AS
        (
         SELECT
             i.customer_id,
             c.country,
             SUM(i.total) total_purchases
         FROM invoice i
         INNER JOIN customer c ON i.customer_id = c.customer_id
         GROUP BY i.customer_id, c.country
        ),
    country_max_purchase AS
        (
         SELECT
             country,
             MAX(total_purchases) max_purchase
         FROM customer_country_purchases
         GROUP BY country
        ),
    country_best_customer AS
        (
         SELECT
            cmp.country,
            cmp.max_purchase,
            (
             SELECT ccp.customer_id
             FROM customer_country_purchases ccp
             WHERE ccp.country = cmp.country AND cmp.max_purchase = ccp.total_purchases
            ) customer_id
         FROM country_max_purchase cmp
        )
SELECT
    cbc.country country,
    c.first_name || " " || c.last_name customer_name,
    cbc.max_purchase total_purchased
FROM customer c
INNER JOIN country_best_customer cbc ON cbc.customer_id = c.customer_id
ORDER BY country ASC
;'''

pd.read_sql_query(q, conn)

,country,customer_name,total_purchased
0,Argentina,Diego Gutiérrez,39.60
1,Australia,Mark Taylor,81.18
2,Austria,Astrid Gruber,69.30
3,Belgium,Daan Peeters,60.39
4,Brazil,Luís Gonçalves,108.90
5,Canada,François Tremblay,99.99
6,Chile,Luis Rojas,97.02
7,Czech Republic,František Wichterlová,144.54
8,Denmark,Kara Nielsen,37.62
9,Finland,Terhi Hämäläinen,79.20
